In [1]:
#Getting the dataset
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

#Calibration dataset (recommended > 1024)
#used to gather activation statistics in the inputs/outputs of each layer
#The dataset is already normalized
calib_dataset = test_images[0:2000]

In [2]:
from hailo_sdk_client import ClientRunner
import os
runner = ClientRunner(hw_arch='hailo8')

model_name = 'mnist'  
hailo_model_har_name = '{}_hailo_model.har'.format(model_name)
assert os.path.isfile(hailo_model_har_name), 'Please provide valid path for HAR file'
runner = ClientRunner(hw_arch='hailo8', har_path=hailo_model_har_name)

#There can be a script to change the model (normalize or resize the input, ...)

#runner.load_model_script('script.alls')

#After the statistics are collected, they are used to quantize the weights and activtions, 
#that is, map the floating point values into integer representation
runner.optimize(calib_dataset)

[info] Starting Model Optimization


2023-04-10 16:28:28.088457: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-10 16:28:28.088491: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-10 16:28:28.088506: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-FNN0JLC): /proc/driver/nvidia/version does not exist
2023-04-10 16:28:28.088709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[info] Using calibration set of 64 entries
[info] Starting Stats Collector


Calibration: 100%|█████████████████████████████████████████████████████████████████| 64/64 [00:02<00:00, 25.71entries/s]

[info] Stats Collector is done (completion time is 00:00:02.60)


[info] No shifts available for layer mnist/fc1/dense_op, using max shift instead. delta=1.2123495673798823
[info] Bias Correction skipped
[info] Adaround skipped
[info] Fine Tune skipped
[info] Model Optimization is done


In [3]:
#Saving quantized har
quantized_model_har_path = '{}_quantized_model.har'.format(model_name)
runner.save_har(quantized_model_har_path)